In [ ]:
# main_app.py (refined UI)
import os
import threading
import customtkinter as ctk
from tkinter import Toplevel, messagebox
import tkinter as tk

# Existing modules
import data_management
import delay_prediction
import cost_optimization

# import the three AI modules by module name 
import prediction_module
import feature_importance_module
import param_optimization_module

# NEW modules
import slope_stability
import back_break
import flyrock   

import joblib
import pandas as pd

# =========================
# Constants
# =========================
INPUT_LABELS = [
    "Hole depth (m)", "Hole diameter (mm)", "Burden (m)", "Spacing (m)",
    "Stemming (m)", "Distance (m)", "Powder factor (kg/m³)", "Rock density (t/m³)",
    "Linear charge (kg/m)", "Explosive mass (kg)", "Blast volume (m³)", "# Holes",
]
OUTPUTS = ["Fragmentation", "Ground Vibration", "Airblast"]

MODEL_FILES = {
    "Fragmentation":    "random_forest_model_Fragmentation.joblib",
    "Ground Vibration": "random_forest_model_Ground Vibration.joblib",
    "Airblast":         "random_forest_model_Airblast.joblib",
}
SCALER_FILE = "scaler1.joblib"

DEFAULT_DATASETS = [
    "combinedv2Orapa.csv",
    "combinedv2Jwaneng.csv",
]

# =========================
# Model/Dataset Registry
# =========================
class ModelRegistry:
    """Thread-safe registry for scaler, models and a default dataset."""
    def __init__(self):
        self._lock = threading.Lock()
        self._scaler = None
        self._models = {}
        self._data = None
        self._data_path = None
        self._loaded = False
        self._load_error = None

    def preload(self):
        if self._loaded:
            return
        with self._lock:
            if self._loaded:
                return
            try:
                # Scaler
                if os.path.exists(SCALER_FILE):
                    self._scaler = joblib.load(SCALER_FILE)
                # Models
                for out_name, fname in MODEL_FILES.items():
                    if os.path.exists(fname):
                        self._models[out_name] = joblib.load(fname)
                # Dataset
                for candidate in DEFAULT_DATASETS:
                    if os.path.exists(candidate):
                        self._data = pd.read_csv(candidate)
                        self._data_path = os.path.abspath(candidate)
                        break
                self._loaded = True
            except Exception as e:
                self._load_error = str(e)
                self._loaded = True

    def status(self):
        with self._lock:
            return self._loaded, self._load_error

    def get_scaler(self):
        with self._lock:
            return self._scaler

    def get_models(self):
        with self._lock:
            return dict(self._models)

    def get_dataset(self):
        with self._lock:
            return self._data, self._data_path

    def set_dataset(self, csv_path: str):
        df = pd.read_csv(csv_path)
        with self._lock:
            self._data = df
            self._data_path = os.path.abspath(csv_path)

    @staticmethod
    def ensure_feature_vector(values: dict):
        import numpy as np
        return np.asarray([float(values.get(lbl, 0.0)) for lbl in INPUT_LABELS], dtype=float).reshape(1, -1)

REGISTRY = ModelRegistry()

# =========================
# Main shell app
# =========================
class ModernBlastingShell(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Blasting Optimization Suite")
        self.geometry("1240x800")
        self.minsize(1060, 700)

        # Global look
        ctk.set_appearance_mode("System")       # or "Dark"
        ctk.set_default_color_theme("blue")     # "blue" | "green" | "dark-blue"

        # ---- Root grid: header (row 0), body (row 1), status bar (row 2)
        self.grid_columnconfigure(0, weight=0)
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(1, weight=1)

        self._sidebar_visible = True
        self._status_lines = []

        self._build_menu()     # still useful (shortcuts & menu lovers)
        self._build_header()   # NEW modern header
        self._build_sidebar()  # Collapsible sidebar
        self._build_home()     # Main area (cards)
        self._build_statusbar()# Thin status bar

        # preload models/dataset
        threading.Thread(target=REGISTRY.preload, daemon=True).start()
        self.after(400, self._poll_status)

        # handy shortcuts
        accel = "Command" if self.tk.call('tk', 'windowingsystem') == 'aqua' else "Control"
        self.bind_all(f"<{accel}-r>", lambda _e: self._reload_registry())
        self.bind_all(f"<{accel}-d>", lambda _e: self._open_data_manager())
        self.bind_all("<F1>", lambda _e: self._about())

    # ---------- Header ----------
    def _build_header(self):
        self.header = ctk.CTkFrame(self, height=56)
        self.header.grid(row=0, column=0, columnspan=2, sticky="nsew")
        self.header.grid_columnconfigure(0, weight=0)
        self.header.grid_columnconfigure(1, weight=1)
        self.header.grid_columnconfigure(2, weight=0)
        self.header.grid_columnconfigure(3, weight=0)
        self.header.grid_columnconfigure(4, weight=0)
        self.header.grid_columnconfigure(5, weight=0)

        # Sidebar toggle
        self._btn_hamburger = ctk.CTkButton(
            self.header, text="☰", width=38, command=self._toggle_sidebar
        )
        self._btn_hamburger.grid(row=0, column=0, padx=(10, 6), pady=10)

        # Title
        self._title = ctk.CTkLabel(
            self.header, text="Blasting Optimization Suite",
            font=ctk.CTkFont(size=18, weight="bold")
        )
        self._title.grid(row=0, column=1, sticky="w", padx=(6, 6))

        # Dataset chip (updates as soon as we know)
        self._dataset_chip = ctk.CTkLabel(
            self.header, text="Dataset: (loading…)",
            corner_radius=12, fg_color=("gray90", "gray20"), text_color=("black", "white"),
            padx=10, pady=6
        )
        self._dataset_chip.grid(row=0, column=2, padx=6)

        # Theme selector
        self._theme = ctk.CTkOptionMenu(self.header, values=["System", "Light", "Dark"],
                                        command=lambda m: ctk.set_appearance_mode(m))
        self._theme.set("System")
        self._theme.grid(row=0, column=3, padx=(6, 6))

        # Accent selector
        self._accent = ctk.CTkOptionMenu(self.header, values=["blue", "green", "dark-blue"],
                                         command=lambda m: ctk.set_default_color_theme(m))
        self._accent.set("blue")
        self._accent.grid(row=0, column=4, padx=(6, 6))

        # Reload button
        self._btn_reload = ctk.CTkButton(self.header, text="Reload", command=self._reload_registry)
        self._btn_reload.grid(row=0, column=5, padx=(6, 10))

    def _toggle_sidebar(self):
        self._sidebar_visible = not self._sidebar_visible
        if self._sidebar_visible:
            self.sidebar.grid(row=1, column=0, sticky="nsw")
            self._btn_hamburger.configure(text="☰")
        else:
            self.sidebar.grid_forget()
            self._btn_hamburger.configure(text="⫶")

    # ---------- Menu (kept simple) ----------
    def _build_menu(self):
        menu = tk.Menu(self)

        theme_menu = tk.Menu(menu, tearoff=0)
        theme_menu.add_command(label="Light", command=lambda: ctk.set_appearance_mode("Light"))
        theme_menu.add_command(label="Dark", command=lambda: ctk.set_appearance_mode("Dark"))
        theme_menu.add_command(label="System", command=lambda: ctk.set_appearance_mode("System"))
        menu.add_cascade(label="Theme", menu=theme_menu)

        data_menu = tk.Menu(menu, tearoff=0)
        data_menu.add_command(label="Open Data Manager\tCtrl+D", command=self._open_data_manager)
        data_menu.add_command(label="Reload Models/Dataset\tCtrl+R", command=self._reload_registry)
        menu.add_cascade(label="Data", menu=data_menu)

        help_menu = tk.Menu(menu, tearoff=0)
        help_menu.add_command(label="About\tF1", command=self._about)
        menu.add_cascade(label="Help", menu=help_menu)

        self.config(menu=menu)

    # ---------- Sidebar (collapsible groups) ----------
    def _build_sidebar(self):
        self.sidebar = ctk.CTkScrollableFrame(self, width=260, corner_radius=0)
        self.sidebar.grid(row=1, column=0, sticky="nsw")
        self.sidebar.grid_columnconfigure(0, weight=1)

        def _group(title, items, start_row):
            g = ctk.CTkCollapsibleFrame(self.sidebar, title)
            # Place the collapsible in the sidebar WITH GRID (this is fine)
            g.grid(row=start_row, column=0, padx=12, pady=(8, 2), sticky="new")
            # IMPORTANT: inside the collapsible, use PACK (not grid) for its children
            for label, cmd in items:
                b = ctk.CTkButton(g, text=label, command=cmd)
                b.pack(fill="x", expand=True, padx=8, pady=5)
            return g

        r = 0
        _group("Analysis", [
            ("📊 Prediction", lambda: prediction_module.PredictionWindow(self, REGISTRY, INPUT_LABELS, OUTPUTS)),
            ("🧭 Feature Importance", lambda: feature_importance_module.FeatureImportanceWindow(self, REGISTRY, INPUT_LABELS, OUTPUTS)),
            ("🧪 Parameter Optimisation", lambda: param_optimization_module.ParamOptimWindow(self, REGISTRY, INPUT_LABELS, OUTPUTS)),
        ], r); r += 1

        _group("Operations", [
            ("💥 Cost Optimisation", lambda: cost_optimization.OptimizationApp(Toplevel(self))),
            ("⏱️ Delay Prediction", self._open_delay_prediction),
        ], r); r += 1

        _group("Safety / Geo", [
            ("🧱 Slope Stability", lambda: slope_stability.SlopeStabilityWindow(Toplevel(self))),
            ("🔧 Back Break",     lambda: back_break.BackbreakWindow(Toplevel(self))),
            ("🪨 Flyrock (ML + Empirical)", lambda: flyrock.FlyrockWindow(Toplevel(self))),
        ], r); r += 1

        _group("Admin", [
            ("🗂️ Data Manager", self._open_data_manager),
        ], r); r += 1

    # ---------- Home (cards) ----------
    def _build_home(self):
        self.home = ctk.CTkFrame(self)
        self.home.grid(row=1, column=1, sticky="nsew", padx=12, pady=12)
        self.home.grid_columnconfigure((0,1,2), weight=1)
        self.home.grid_rowconfigure(2, weight=1)

        title = ctk.CTkLabel(self.home, text="Welcome 👋",
                             font=ctk.CTkFont(size=22, weight="bold"))
        title.grid(row=0, column=0, columnspan=3, sticky="w", pady=(4,8))

        subtitle = ctk.CTkLabel(
            self.home,
            text=("AI-driven blast design • Cost & constraint-aware optimisation • "
                  "USBM + Kuz–Ram empirical baselines"),
            text_color=("gray20","gray80")
        )
        subtitle.grid(row=1, column=0, columnspan=3, sticky="w", pady=(0,10))

        def card(col, title, desc, command):
            f = ctk.CTkFrame(self.home, corner_radius=16)
            f.grid(row=2, column=col, sticky="nsew", padx=6, pady=6)
            f.grid_rowconfigure(2, weight=1)
            ctk.CTkLabel(f, text=title, font=ctk.CTkFont(size=16, weight="bold")).grid(row=0, column=0, sticky="w", padx=14, pady=(14,4))
            ctk.CTkLabel(f, text=desc, wraplength=320, justify="left").grid(row=1, column=0, sticky="w", padx=14)
            ctk.CTkButton(f, text="Open", command=command).grid(row=3, column=0, sticky="we", padx=14, pady=14)

        card(0, "📊 Prediction",
             "Run ML & empirical predictions (USBM PPV/Air, Kuz–Ram Xm + RR curve).",
             lambda: prediction_module.PredictionWindow(self, REGISTRY, INPUT_LABELS, OUTPUTS))
        card(1, "💥 Cost Optimisation",
             "Minimise cost with penalties for PPV, airblast, fragmentation (Xm→RR X50).",
             lambda: cost_optimization.OptimizationApp(Toplevel(self)))
        card(2, "🪨 Flyrock",
             "Predict flyrock (ML + empirical lines); check limits and distances.",
             lambda: flyrock.FlyrockWindow(Toplevel(self)))

    # ---------- Status bar ----------
    def _build_statusbar(self):
        self.statusbar = ctk.CTkFrame(self, height=28, corner_radius=0)
        self.statusbar.grid(row=2, column=0, columnspan=2, sticky="ew")
        self.statusbar.grid_columnconfigure(0, weight=1)
        self._status_lbl = ctk.CTkLabel(self.statusbar, text="Starting…")
        self._status_lbl.grid(row=0, column=0, sticky="w", padx=10, pady=4)
        self._status_prog = ctk.CTkProgressBar(self.statusbar, mode="indeterminate", width=160)
        self._status_prog.grid(row=0, column=1, sticky="e", padx=10, pady=6)
        self._status_prog.start()

    # ---- actions ----
    def _open_data_manager(self):
        data_management.DataApp(Toplevel(self))

    def _open_delay_prediction(self):
        top = Toplevel(self)
        app = delay_prediction.BlastSimApp(top)
        def _on_close():
            try:
                pass
            finally:
                top.destroy()
        top.protocol("WM_DELETE_WINDOW", _on_close)

    def _reload_registry(self, *_):
        def _reload():
            try:
                self._set_status("Reloading models/scaler/dataset…")
                self._status_prog.start()
                global REGISTRY
                REGISTRY = ModelRegistry()
                REGISTRY.preload()
                self._append_status("Registry reloaded.")
            except Exception as e:
                self._append_status(f"Reload error: {e}")
            finally:
                self._status_prog.stop()
        threading.Thread(target=_reload, daemon=True).start()

    def _poll_status(self):
        ready, err = REGISTRY.status()
        if not ready:
            self._set_status("Loading models/scaler/dataset…")
            self.after(700, self._poll_status)
            return

        self._status_prog.stop()
        if err:
            self._set_status(f"Load error: {err}")
            self._dataset_chip.configure(text="Dataset: (error)")
        else:
            _, path = REGISTRY.get_dataset()
            if path:
                base = os.path.basename(path)
                self._set_status(f"Ready. Dataset: {base}")
                self._dataset_chip.configure(text=f"Dataset: {base}")
            else:
                self._set_status("Ready. No dataset attached yet.")
                self._dataset_chip.configure(text="Dataset: (none)")

    # ---------- small status helpers ----------
    def _append_status(self, msg: str):
        self._status_lines.append(msg)
        self._status_lbl.configure(text=msg)

    def _set_status(self, msg: str):
        self._status_lbl.configure(text=msg)

    def _about(self):
        messagebox.showinfo(
            "About",
            "Blasting Optimization Suite — Modular Shell\n"
            "AI predictions • Empirical USBM/Kuz–Ram • Cost & constraints\n"
            "Delay • Slope Stability • Back Break • Flyrock\n"
            "© 2025"
        )


if __name__ == "__main__":
    app = ModernBlastingShell()
    app.mainloop()
